## Spartans_AI-Classifican-Model-for-Diabetic-Retinopathy--DR-
- Tauha Imran
- Moaz Farooq
- Sarim Rasheed

In [ ]:
# ONLY RUN IF DATA NOT DOWNLOADED!!!
# RUN THIS CELL ONLY ONCE!!!

#if dependency issue
# pip install kagglehub

# Importing TensorFlow - dataset
import kagglehub
# Download latest version
path = kagglehub.dataset_download("kushagratandon12/diabetic-retinopathy-balanced")

print("Path to dataset files:", path)

C:\Users\tauha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  1%|          | 23.0M/1.90G [00:11<13:34, 2.47MB/s]

The actual model training code

In [6]:


# Importing TensorFlow
import tensorflow as tf

# Importing PyTorch
import torch

# Importing OpenCV
import cv2

# Importing FastAI
#from fastai.vision.all import *

# Importing Scikit-Learn
import sklearn

print('testing')


testing
